In [ ]:
"""
We want to take a look at using the AST library.
Below is a simple function that we will be examining using ASTs.
"""
import inspect

def calculate_average(numbers):
    total = 0
    for element in numbers:
        total += element
    average = total / len(numbers)
    print(average)
    return average

code = inspect.getsource(calculate_average)
print("=== code ===")
print(code)

As far as using the AST goes, some of the basic features we would want to satisfy are: 
* To be able to pass the source code above into an AST and iterate through it, node by node
* At each node, to be able to get some meaning out of the type of node we're looking at (e.g. does this node represent a 'for loop'?)
* Supposing we had a list of rules similar to those we created for structural explanations. Could we check to see that the rules are met in order as we proceed through the ast walk?

As an end goal, try to write a program that checks for the existence of a 'for loop', a 'print statement', and a 'return statement', in that order.

Some resources to get you started:
(but by no means limit yourself to these; look for more, for ones that you understand best)
* https://www.youtube.com/watch?v=OjPT15y2EpE
* https://medium.com/@wshanshan/intro-to-python-ast-module-bbd22cd505f7
* https://earthly.dev/blog/python-ast/
* https://docs.python.org/3/library/ast.html

In [ ]:
import ast
import pprint

tree = ast.parse(code)
# pprint.pprint(ast.dump(tree))

print("=== ast walk ===")

"""
Here we walk through the ast, going 1 node at a time.
The nodes do not correspond one-to-one with each line of code, but they are still linked in meaning.
Try to read through the printout of the walk and see how each node in the walk corresponds to an element of the given code.
(In particular, the 'lineno' or 'line number' field in the printout should be helpful for understanding the connection.)
"""
for node in ast.walk(tree):
    print(node)
    print(node.__dict__)
    # print(node._fields)
    print("children: " + str([x for x in ast.iter_child_nodes(node)]) + "\n")

    # How could we check to see if it is a certain type of node?
    # ...

    # If we were to go further than that, suppose we had a list of rules similar to those we created for structural explanations...
    # Could we check to see that the rules are met in order as we proceed through the ast walk?
    # ...

In [ ]:
"""
Some older code you might find useful as a reference.
This method uses the 'NodeVisitor' approach to move through the AST, different from the ast walk approach seen above.

These are magic functions, which are called depending on the name.
The 'visit_Call()' function is called on all nodes that are of 'call' type
The 'visit_For()' function is called on all nodes that are of 'for' type
"""
class FunctionCallVisitor(ast.NodeVisitor):
    def visit_Call(self, node):
        if isinstance(node.func, ast.Name) and node.func.id == "print":
            print(f"yup found a line with a print. It is this line: {node.__dict__}")
            args = [arg for arg in node.args if isinstance(arg, ast.Constant)]
            if args:
                print("Detected print statements with string literals:")
                for arg in args:
                    print(arg.s)  # Print the string literal directly
        self.generic_visit(node)

    def visit_For(self, node):
        # if isinstance(node.func, ast.Name) and node.func.id == "print":
        # print(f"args = {node.args}")
        # args = [arg for arg in node.args if isinstance(arg, ast.Constant)]
        # print(f"node.func = {node.func}")
        # args = [arg for arg in node.args if isinstance(arg, ast.For)]
        # if args:

        # class ast.For(target, iter, body, orelse, type_comment)
        # https://docs.python.org/3/library/ast.html > ctrl+f > ast.for

        print("Detected For statement with...")
        print(f"node.target={node.target} \nnode.iter={node.iter} \nnode.body={node.body} \nnode.orelse={node.orelse} \nnode.type_comment={node.type_comment}")

def perform_static_analysis(code):
    tree = ast.parse(code)
    visitor = FunctionCallVisitor()
    visitor.visit(tree)

perform_static_analysis(code)